---

In [19]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

Found 60600 validated image filenames.
Found 15150 validated image filenames.


In [23]:


def create_model(input_shape, num_categories, num_dishes):
    inputs = Input(shape=input_shape)


    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)


    food_category_output = Dense(num_categories, activation='softmax', name='food_category')(x)
    

    dish_output = Dense(num_dishes, activation='softmax', name='dish')(x)


    model = Model(inputs=inputs, outputs=[food_category_output, dish_output])

    return model


input_shape = (128, 128, 3)  
num_categories = 3  
num_dishes = 101  


model = create_model(input_shape, num_categories, num_dishes)


model.compile(optimizer='adam',
              loss={'food_category': 'sparse_categorical_crossentropy', 'dish': 'sparse_categorical_crossentropy'},
              metrics=['accuracy'])


model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv2d_18 (Conv2D)          (None, 128, 128, 32)         896       ['input_2[0][0]']             
                                                                                                  
 max_pooling2d_18 (MaxPooli  (None, 64, 64, 32)           0         ['conv2d_18[0][0]']           
 ng2D)                                                                                            
                                                                                                  
 conv2d_19 (Conv2D)          (None, 64, 64, 64)           18496     ['max_pooling2d_18[0][0]

In [24]:

data = pd.read_csv('train.csv')

train_df, valid_df = train_test_split(data, test_size=0.2, random_state=42)


train_datagen = ImageDataGenerator(
    rescale=1./255,   
    rotation_range=20,  
    width_shift_range=0.2,  
    height_shift_range=0.2,  
    shear_range=0.2,   
    zoom_range=0.2,   
    horizontal_flip=True,  
    fill_mode='nearest' 
)


valid_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='./',  
    x_col='path',
    y_col=['food_category', 'dish'],
    target_size=(128, 128),  
    batch_size=32,
    class_mode='multi_output',
    shuffle=True
)


valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid_df,
    directory='./',
    x_col='path',
    y_col=['food_category', 'dish'],
    target_size=(128, 128),
    batch_size=32,
    class_mode='multi_output',
    shuffle=True
)


history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=10, 
    validation_data=valid_generator,
    validation_steps=valid_generator.n // valid_generator.batch_size
)

Found 60600 validated image filenames.
Found 15150 validated image filenames.
Epoch 1/10
1893/1893 [==============================] - 1257s 662ms/step - loss: 5.0552 - food_category_loss: 0.6376 - dish_loss: 4.4176 - food_category_accuracy: 0.7462 - dish_accuracy: 0.0362 - val_loss: 4.6958 - val_food_category_loss: 0.5880 - val_dish_loss: 4.1078 - val_food_category_accuracy: 0.7513 - val_dish_accuracy: 0.0796
Epoch 2/10
1893/1893 [==============================] - 1134s 598ms/step - loss: 4.6810 - food_category_loss: 0.5727 - dish_loss: 4.1083 - food_category_accuracy: 0.7620 - dish_accuracy: 0.0750 - val_loss: 4.4535 - val_food_category_loss: 0.5646 - val_dish_loss: 3.8889 - val_food_category_accuracy: 0.7656 - val_dish_accuracy: 0.1083
Epoch 3/10
1893/1893 [==============================] - 1168s 617ms/step - loss: 4.4843 - food_category_loss: 0.5465 - dish_loss: 3.9378 - food_category_accuracy: 0.7695 - dish_accuracy: 0.1004 - val_loss: 4.3411 - val_food_category_loss: 0.5208 - val_

In [25]:
test_df = pd.read_csv('test.csv')


test_datagen = ImageDataGenerator(rescale=1./255)


test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='./',  
    x_col='path',
    y_col=None,  
    target_size=(128, 128),  
    batch_size=32,  
    class_mode=None,  
    shuffle=False
)


predictions = model.predict(test_generator)


category_predictions = predictions[0]
dish_predictions = predictions[1]


category_labels = category_predictions.argmax(axis=1)
dish_labels = dish_predictions.argmax(axis=1)


test_df['predicted_category'] = category_labels
test_df['predicted_dish'] = dish_labels

Found 25250 validated image filenames.
790/790 [==============================] - 116s 145ms/step


In [26]:
test_df

,path,predicted_category,predicted_dish
0,images/caprese_salad/820512.jpg,2,42
1,images/hamburger/2062556.jpg,2,95
2,images/lasagna/2381454.jpg,2,25
3,images/ceviche/1995.jpg,2,42
4,images/poutine/892377.jpg,2,27
...,...,...,...
25245,images/ravioli/2087725.jpg,2,9
25246,images/ice_cream/504131.jpg,2,79
25247,images/cannoli/1187751.jpg,2,15
25248,images/pancakes/477236.jpg,2,40


In [28]:
import pandas as pd

test_df['classes'] = test_df['predicted_category'].astype(str) + ' ' + test_df['predicted_dish'].astype(str)

df_result = test_df[['path', 'classes']]

In [29]:
df_result

,path,classes
0,images/caprese_salad/820512.jpg,2 42
1,images/hamburger/2062556.jpg,2 95
2,images/lasagna/2381454.jpg,2 25
3,images/ceviche/1995.jpg,2 42
4,images/poutine/892377.jpg,2 27
...,...,...
25245,images/ravioli/2087725.jpg,2 9
25246,images/ice_cream/504131.jpg,2 79
25247,images/cannoli/1187751.jpg,2 15
25248,images/pancakes/477236.jpg,2 40


In [31]:
df_result.to_csv("predictions.csv", index=False)

In [33]:
model.save('mi_modelo.keras')